# Alternative models

# SNGAN (WORKING)

In [2]:
from keras import backend as K
from keras.engine import *
#from keras.legacy import interfaces
from keras import activations
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.utils.generic_utils import func_dump
from keras.utils.generic_utils import func_load
from keras.utils.generic_utils import deserialize_keras_object
from keras.utils.generic_utils import has_arg
from keras.utils import conv_utils
#from keras.legacy import interfaces
from keras.layers import Dense, Conv1D, Conv2D, Conv3D, Conv2DTranspose, Embedding
import tensorflow as tf

# Spectral Normalization layer code from: https://github.com/IShengFang/SpectralNormalizationKeras/blob/master/SpectralNormalizationKeras.py
class DenseSN(Dense):
    def build(self, input_shape):
        assert len(input_shape) >= 2
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(shape=(input_dim, self.units),
                                      initializer=self.kernel_initializer,
                                      name='kernel',
                                      regularizer=self.kernel_regularizer,
                                      constraint=self.kernel_constraint)
        if self.use_bias:
            self.bias = self.add_weight(shape=(self.units,),
                                        initializer=self.bias_initializer,
                                        name='bias',
                                        regularizer=self.bias_regularizer,
                                        constraint=self.bias_constraint)
        else:
            self.bias = None
        self.u = self.add_weight(shape=tuple([1, self.kernel.shape.as_list()[-1]]),
                                 initializer=initializers.RandomNormal(0, 1),
                                 name='sn',
                                 trainable=False)
        self.input_spec = InputSpec(min_ndim=2, axes={-1: input_dim})
        self.built = True
        
    def call(self, inputs, training=None):
        def _l2normalize(v, eps=1e-12):
            return v / (K.sum(v ** 2) ** 0.5 + eps)
        def power_iteration(W, u):
            _u = u
            _v = _l2normalize(K.dot(_u, K.transpose(W)))
            _u = _l2normalize(K.dot(_v, W))
            return _u, _v
        W_shape = self.kernel.shape.as_list()
        #Flatten the Tensor
        W_reshaped = K.reshape(self.kernel, [-1, W_shape[-1]])
        _u, _v = power_iteration(W_reshaped, self.u)
        #Calculate Sigma
        sigma=K.dot(_v, W_reshaped)
        sigma=K.dot(sigma, K.transpose(_u))
        #normalize it
        W_bar = W_reshaped / sigma
        #reshape weight tensor
        if training in {0, False}:
            W_bar = K.reshape(W_bar, W_shape)
        else:
            with tf.control_dependencies([self.u.assign(_u)]):
                 W_bar = K.reshape(W_bar, W_shape)  
        output = K.dot(inputs, W_bar)
        if self.use_bias:
            output = K.bias_add(output, self.bias, data_format='channels_last')
        if self.activation is not None:
            output = self.activation(output)
        return output 

In [3]:
class GAN(): # Training based on https://github.com/eriklindernoren/Keras-GAN/blob/master/gan/gan.py
    
    def __init__(self, train, test, numerical_col_n, categorical_col_n, categories_n, categories_cum, # Data
                     intermediate_dim_gen=256, latent_dim=100, n_hidden_layers_gen=4, # Architecture Generator
                     intermediate_dim_dis=128, n_hidden_layers_dis=2, # Architecture Generator
                     batch_size=64, epochs=50, validation_split=0.2, gen_learn_rate=0.001, dis_learn_rate=0.001): # Training
        
        # Data parameters
        self.data_train = train
        self.data_test = test
        
        self.numerical_col_n = numerical_col_n # Scalar
        self.categorical_col_n = categorical_col_n # Scalar
        self.categories_n = categories_n # List of scalars
        self.categories_cum = categories_cum # List of scalars
        
        # Architecture parameters
        self.input_dim_dis = train.shape[1]
        self.latent_dim = latent_dim # Input dimension for generator
        self.intermediate_dim_dis = intermediate_dim_dis
        self.intermediate_dim_gen = intermediate_dim_gen
        self.n_hidden_layers_dis = n_hidden_layers_dis
        self.n_hidden_layers_gen = n_hidden_layers_gen
        
        # Training parameters
        self.epochs = epochs
        self.batch_size = batch_size
        self.validation_split = validation_split
        self.dis_learn_rate = dis_learn_rate
        self.gen_learn_rate = gen_learn_rate
        self.dis_opt = keras.optimizers.Adam(lr=self.dis_learn_rate)
        self.gen_opt = keras.optimizers.Adam(lr=self.gen_learn_rate)
        self.loss = 'binary_crossentropy'
        
        # Sampling parameters
        self.n_samples = test.shape[0]
        
        # Model creation
        self.create_gan()
        
        # Session variables
        #tf.reset_default_graph()
        sess = tf.InteractiveSession() # Start tf session so we can run code.
        K.set_session(sess) # Connect keras to the created session.
        

    # Generator architecture
    def create_generator(self):
        '''
        This is the generator architecture, 

        params: 
        n_hidden_layers_gen: number of hidden layers
        intermediate_dim_gen: value of the number of neurons for the first intermediate layer. 
            They increase in a factor of 2 (N*2). 
        latent_dim: dimension of latent space taken as input for the generator
        
        output:
        The function outputs the generator model in keras. The architecture is defined by the user when
        the whole GAN class is initialized.
        '''
            
        gen_input = Input(shape=(self.latent_dim,), name='gen_input')
        
        # Make the generator intermediate dimensions as it should be first
        self.intermediate_dim_gen = int(self.intermediate_dim_gen/2**(self.n_hidden_layers_gen-1))
        
        # Intermediate layers
        for _ in range(self.n_hidden_layers_gen):
            if _==0: # The first one takes the inputs as input
                intermediate = Dense(self.intermediate_dim_gen, name= 'generator_hidden_{}'.format(_), kernel_initializer='he_uniform', activation='relu')(gen_input)
            else: # After the first one, the network takes the intermediate layers as input
                intermediate = Dense(self.intermediate_dim_gen, name= 'generator_hidden_{}'.format(_), kernel_initializer='he_uniform', activation='relu')(intermediate)
            self.intermediate_dim_gen *= 2 # Update the value of the number of neurons

        # Final layer
        # Categorical decode
        x_decoded_mean_cat = [Dense(categories_n[cat], activation='softmax')(intermediate) 
                              for cat in range(len(self.categories_n))]

        if numerical_col_n > 0: # If there are numerical variables, concatenate both
            x_decoded_mean_num = Dense(self.numerical_col_n)(intermediate) # Numerical decode
            gen_output = concatenate([x_decoded_mean_num] + x_decoded_mean_cat, name='gen_output')
        else: # If there are no numerical variables only include the categorical output layer
            gen_output = concatenate(x_decoded_mean_cat, name='gen_output')

        return Model(inputs=gen_input, outputs=gen_output)
    
    # Discriminator architecture
    def create_discriminator(self):
        '''
        COMMENT THE CODE
        '''

        dis_input = Input(shape=(self.input_dim_dis,), name='dis_input')
        
        # Intermediate layers
        for _ in range(self.n_hidden_layers_dis):
            if _==0: # The first one takes the inputs as input
                intermediate = DenseSN(self.intermediate_dim_dis, name= 'discriminator_hidden_{}'.format(_), kernel_initializer='he_uniform', activation='relu')(dis_input)
            else: # After the first one, the network takes the intermediate layers as input
                intermediate = DenseSN(self.intermediate_dim_dis, name= 'discriminator_hidden_{}'.format(_), kernel_initializer='he_uniform', activation='relu')(intermediate)
            self.intermediate_dim = int(self.intermediate_dim_dis/2) # Update the value of the number of neurons
        
        dis_output = Dense(1, activation='sigmoid', name='dis_output')(intermediate)
        
        return Model(dis_input, dis_output)
    
    # GAN creation
    def create_gan(self):
        '''
        COMMENT THE CODE
        '''
        
        # Build and compile the discriminator
        self.discriminator = self.create_discriminator()
        self.discriminator.compile(loss=self.loss, optimizer=self.dis_opt, metrics=['accuracy'])

        # Build the generator
        self.generator = self.create_generator()

        # The generator takes noise as input and generates observations
        z = Input(shape=(self.latent_dim,))
        generated = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated observations as input and discriminates
        guess = self.discriminator(generated)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.gan = Model(z, guess)
        self.gan.compile(loss=self.loss, optimizer=self.gen_opt) 
        

    def gan_fit(self):
        '''
        COMMENT THE CODE
        '''

        # Adversarial ground truths
        valid = np.ones((self.batch_size, 1))
        fake = np.zeros((self.batch_size, 1))

        # Save the generator and discriminator losses and accuracies
        self.gen_loss = []
        self.dis_loss = []
        self.dis_acc = []
        
        for epoch in range(self.epochs):
            
            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of observations
            idx = np.random.choice(self.data_train.shape[1], self.batch_size, replace=False)
            obs = self.data_train[idx]

            noise = np.random.normal(0, 1, (self.batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_obs = self.generator.predict(noise)

            # Train the discriminator
            dis_loss_real = self.discriminator.train_on_batch(obs, valid)
            dis_loss_fake = self.discriminator.train_on_batch(gen_obs, fake)
            self.dis_loss.append(0.5 * np.add(dis_loss_real[0], dis_loss_fake[0]))
            self.dis_acc.append(0.5 * np.add(dis_loss_real[1], dis_loss_fake[1]))

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (self.batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            self.gen_loss.append(self.gan.train_on_batch(noise, valid))
            
            if epoch%100==0:
                # Plot the progress
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, self.dis_loss[-1], 100*self.dis_acc[-1], self.gen_loss[-1]))
                #print (dis_loss_real[0], dis_loss_fake[0], gen_loss)
                #print(gen_obs[1,:], np.sum(gen_obs, axis=1))

    # Sampling helper function for evaluation
    def sampler(self):
        z_sample = np.random.normal(0., 1.0, size=(self.n_samples, self.latent_dim))
        prediction = self.generator.predict(z_sample).transpose()
        samples = np.zeros((self.input_dim_dis, self.n_samples))
        samples[:self.numerical_col_n,:]=prediction[:self.numerical_col_n,:]
        for idx in range(len(self.categories_cum)-1):
            idx_i = self.numerical_col_n+self.categories_cum[idx] # Initial index
            idx_f = self.numerical_col_n+self.categories_cum[idx+1] # Final index
            mask = np.argmax(prediction[idx_i:idx_f, :], axis=0) + idx_i
            for n in range(self.n_samples):
                samples[mask[n], n] = 1
        return samples
    
    # GAN evaluation
    def gan_evaluate(self, used_metric='MAE'):
        '''
        COMMENT THE CODE
        '''
        # Fit the model
        self.gan_fit()
        
        # Evaluate it
        self.samples = self.sampler()
        self.gan_df = samples_to_df(self.samples, print_duplicates=False)
        test_df = samples_to_df(self.data_test.transpose(), print_duplicates=False)

        # Numerical bin creator 
        for var in numerical:
            test_df[var], bins = pd.cut(test_df[var], bins=5,  retbins=True)
            self.gan_df[var] = pd.cut(self.gan_df[var], bins=bins)

        agg_vars = categorical # Variables we are using to aggregate and evaluate, change as needed 
        ##### Count creator
        self.gan_df['count'] = 1
        self.gan_df = self.gan_df.groupby(agg_vars, observed=True).count()
        self.gan_df /= self.gan_df['count'].sum()

        test_df['count'] = 1
        test_df = test_df.groupby(agg_vars, observed=True).count()
        test_df /= test_df['count'].sum()

        ##### Merge and difference
        real_and_sampled = pd.merge(test_df, self.gan_df, suffixes=['_real', '_sampled'], on=categorical, how='outer') # on= all variables
        real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)
        real_and_sampled['diff'] = real_and_sampled.count_real-real_and_sampled.count_sampled
        diff = np.array(real_and_sampled['diff'])
        
        metrics = {}
        metrics['MAE'] = np.mean(abs(diff))
        metrics['MSE'] = np.mean(diff**2)
        metrics['RMSE'] = np.sqrt(np.mean(diff**2))
        print('Evaluating with {}'.format(used_metric))
        print('MAE:{}, MSE:{}, RMSE:{}'.format(metrics['MAE'], metrics['MSE'], metrics['RMSE']))  
        
        return metrics[used_metric]

# WGAN GP (NOT WORKING)

In [19]:
# WGAN-GP
class RandomWeightedAverage(_Merge):
        """
        Provides a (random) weighted average between real and generated image samples
        """
        def _merge_function(self, inputs):
            alpha = K.random_uniform((64, 1)) # (batch_size, 1)
            return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])
        
class WGANGP(): # Training partly based on https://github.com/eriklindernoren/Keras-GAN/blob/master/gan/gan.py
    
    def __init__(self, train, test, numerical_col_n, categorical_col_n, categories_n, categories_cum, # Data
                     intermediate_dim_gen=256, latent_dim=100, n_hidden_layers_gen=4, # Architecture Generator
                     intermediate_dim_crit=128, n_hidden_layers_crit=2, # Architecture Critic
                     batch_size=64, epochs=50, validation_split=0.2, gen_learn_rate=0.001, crit_learn_rate=0.00005, # Training
                     lmda=10, nCritic=5): # Training
        
        # Data parameters
        self.data_train = train
        self.data_test = test
        
        self.numerical_col_n = numerical_col_n # Scalar
        self.categorical_col_n = categorical_col_n # Scalar
        self.categories_n = categories_n # List of scalars
        self.categories_cum = categories_cum # List of scalars
        
        # Architecture parameters
        self.input_dim_crit = train.shape[1]
        self.latent_dim = latent_dim # Input dimension for generator
        self.intermediate_dim_crit = intermediate_dim_crit
        self.intermediate_dim_gen = intermediate_dim_gen
        self.n_hidden_layers_crit = n_hidden_layers_crit
        self.n_hidden_layers_gen = n_hidden_layers_gen
        
        # Training parameters
        self.epochs = epochs
        self.batch_size = batch_size
        self.validation_split = validation_split
        self.gen_learn_rate = gen_learn_rate
        self.crit_learn_rate = crit_learn_rate
        self.lmda = lmda # WGAN-GP parameter
        self.nCritic = nCritic # WGAN parameter
        self.crit_opt = keras.optimizers.RMSprop(lr=self.crit_learn_rate)
        self.gen_opt = keras.optimizers.RMSprop(lr=self.gen_learn_rate) #keras.optimizers.Adam(lr=self.gen_learn_rate)
        
        # Sampling parameters
        self.n_samples = test.shape[0]
        
        # Model creation
        self.create_wgangp()
        
        # Session variables
        #tf.reset_default_graph()
        sess = tf.InteractiveSession() # Start tf session so we can run code.
        K.set_session(sess) # Connect keras to the created session.
        

    # Generator architecture
    def create_generator(self):
        '''
        This is the generator architecture, 

        params: 
        n_hidden_layers_gen: number of hidden layers
        intermediate_dim_gen: value of the number of neurons for the first intermediate layer. 
            They increase in a factor of 2 (N*2). 
        latent_dim: dimension of latent space taken as input for the generator
        
        output:
        The function outputs the generator model in keras. The architecture is defined by the user when
        the whole GAN class is initialized.
        '''
            
        gen_input = Input(shape=(self.latent_dim,), name='gen_input')
        
        # Make the generator intermediate dimensions as it should be first
        self.intermediate_dim_gen = int(self.intermediate_dim_gen/2**(self.n_hidden_layers_gen-1))
        
        # Intermediate layers
        for _ in range(self.n_hidden_layers_gen):
            if _==0: # The first one takes the inputs as input
                intermediate = Dense(self.intermediate_dim_gen, name= 'generator_hidden_{}'.format(_), kernel_initializer='he_uniform')(gen_input)
                #intermediate = BatchNormalization()(intermediate)
                intermediate = Activation('relu')(intermediate)
                #intermediate = Dropout(rate=0.1)(intermediate)
            else: # After the first one, the network takes the intermediate layers as input
                intermediate = Dense(self.intermediate_dim_gen, name= 'generator_hidden_{}'.format(_), kernel_initializer='he_uniform')(intermediate)
                #intermediate = BatchNormalization()(intermediate)
                intermediate = Activation('relu')(intermediate)
                #intermediate = Dropout(rate=0.1)(intermediate)
            self.intermediate_dim_gen *= 2 # Update the value of the number of neurons

        # Final layer
        # Categorical decode
        x_decoded_mean_cat = [Dense(categories_n[cat], activation='softmax')(intermediate) 
                              for cat in range(len(self.categories_n))]

        if numerical_col_n > 0: # If there are numerical variables, concatenate both
            x_decoded_mean_num = Dense(self.numerical_col_n)(intermediate) # Numerical decode
            gen_output = concatenate([x_decoded_mean_num] + x_decoded_mean_cat, name='gen_output')
        else: # If there are no numerical variables only include the categorical output layer
            gen_output = concatenate(x_decoded_mean_cat, name='gen_output')

        return Model(inputs=gen_input, outputs=gen_output)
    
    # Critic architecture
    def create_critic(self):
        '''
        COMMENT THE CODE
        '''

        crit_input = Input(shape=(self.input_dim_crit,), name='crit_input')
        
        # Intermediate layers
        for _ in range(self.n_hidden_layers_crit):
            if _==0: # The first one takes the inputs as input
                intermediate = Dense(self.intermediate_dim_crit, name= 'critic_hidden_{}'.format(_), kernel_initializer='he_uniform')(crit_input)
                #intermediate = BatchNormalization()(intermediate)
                intermediate = Activation('relu')(intermediate)
                #intermediate = Dropout(rate=0.3)(intermediate)
            else: # After the first one, the network takes the intermediate layers as input
                intermediate = Dense(self.intermediate_dim_crit, name= 'critic_hidden_{}'.format(_), kernel_initializer='he_uniform')(intermediate)
                #intermediate = BatchNormalization()(intermediate)
                intermediate = Activation('relu')(intermediate)
                #intermediate = Dropout(rate=0.3)(intermediate)
            self.intermediate_dim_crit = int(self.intermediate_dim_crit/2) # Update the value of the number of neurons
        
        crit_output = Dense(1, name='crit_output')(intermediate)
        
        return Model(crit_input, crit_output)
    
    
    # GAN creation
    def create_wgangp(self):
        '''
        COMMENT THE CODE
        '''
        # Create the critic and the generator
        self.critic = self.create_critic()
        self.generator = self.create_generator()
        
        #-------------------------------
        # Construct Computational Graph
        #       for the Critic
        #-------------------------------

        # Freeze generator's layers while training critic
        self.generator.trainable = False

        # Real sample input
        real_obs = Input(shape=(self.input_dim_crit,))
        
        # Noise input
        z_disc = Input(shape=(self.latent_dim,))
        # Generate sample based of noise (fake sample)
        gen_obs = self.generator(z_disc)

        # Discriminator determines validity of the real and fake samples
        fake = self.critic(gen_obs)
        valid = self.critic(real_obs)

        # Construct weighted average between real and fake images
        interpolated = RandomWeightedAverage()([real_obs, gen_obs])
        #alpha = K.random_uniform((self.batch_size, 1))
        #interpolated = tf.keras.layers.Add()([(1 - alpha)*gen_fake, alpha*obs])
    
        # Determine validity of weighted sample
        validity_interpolated = self.critic(interpolated)

        # Use Python partial to provide loss function with additional
        # 'averaged_samples' argument
        partial_gp_loss = partial(self.gradient_penalty_loss, averaged_samples=interpolated)
        partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

        self.critic = Model(inputs=[gen_obs, z_disc], outputs=[valid, fake, validity_interpolated])
        self.critic.compile(loss=[self.wasserstein_loss, self.wasserstein_loss, partial_gp_loss], optimizer=self.crit_opt, loss_weights=[1, 1, self.lmda], metrics=['accuracy']) 

        #-------------------------------
        # Construct Computational Graph
        #         for Generator
        #-------------------------------

        # For the generator we freeze the critic's layers
        self.critic.trainable = False
        self.generator.trainable = True

        # Sampled noise for input to generator
        z_gen = Input(shape=(self.latent_dim,))
        # Generate images based of noise
        generated = self.generator(z_gen)
        print(K.int_shape(generated))
        # Discriminator determines validity
        validation_on_generated = self.critic(generated)
        # Defines generator model
        self.generator = Model(z_gen, validation_on_generated)
        self.generator.compile(loss=self.wasserstein_loss, optimizer=self.gen_opt)
        
        print(self.generatory.summary)
        print(self.critic.summary)


    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
        """
        Computes gradient penalty based on prediction and weighted real / fake samples
        """
        gradients = K.gradients(y_pred, averaged_samples)[0]
        # compute the euclidean norm by squaring ...
        gradients_sqr = K.square(gradients)
        #   ... summing over the rows ...
        gradients_sqr_sum = K.sum(gradients_sqr, axis=np.arange(1, len(gradients_sqr.shape)))
        #   ... and sqrt
        gradient_l2_norm = K.sqrt(gradients_sqr_sum)
        # compute lambda * (1 - ||grad||)^2 still for each single sample
        gradient_penalty = K.square(1 - gradient_l2_norm)
        # return the mean as loss over all the batch samples
        return K.mean(gradient_penalty)
        
    def wasserstein_loss(self, y_true, y_pred):
        '''
        COMMENT THE CODE
        '''
        return K.mean(y_true * y_pred)

    
    def wgangp_fit(self):
        '''
        COMMENT THE CODE
        '''
        
        # Adversarial ground truths
        valid = -np.ones((self.batch_size, 1))
        fake  =  np.ones((self.batch_size, 1))
        dummy =  np.zeros((self.batch_size, 1)) # Dummy gt for gradient penalty
        
        # Loss and accuracy lists for graphing purposes
        self.gen_loss  = []
        self.crit_loss = []
        self.crit_acc  = []
        
        for epoch in range(self.epochs):
            
            # ---------------------
            #  Train Critic
            # ---------------------

            for _ in range(self.nCritic):
                # Select a random batch of observations
                idx = np.random.choice(self.data_train.shape[0], self.batch_size, replace=False) #self.data_train.shape[0]
                obs = self.data_train[idx]

                noise = np.random.normal(0, 1, (self.batch_size, self.latent_dim))

                # Generate a batch of new images
                gen_obs = self.generator.predict(noise)

                # Train the discriminator
                crit_loss = self.critic.train_on_batch([obs, valid], [gen_obs, fake, dummy])
                
                # Clip the weights
                #for l in self.critic.layers:
                #    weights = l.get_weights()
                #    weights = [np.clip(w, -self.clip_value, self.clip_value) for w in weights]
                #    l.set_weights(weights)
                    
            self.crit_loss.append(crit_loss[0])
            self.crit_acc.append(crit_loss[1])

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (self.batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            gen_loss = self.wgan.train_on_batch(noise, valid)
            self.gen_loss.append(gen_loss)
            
            if epoch%100==0:
                # Plot the progress
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, self.crit_loss[-1], 100*self.crit_acc[-1], self.gen_loss[-1]))
                #print (crit_loss_real[0], crit_loss_fake[0], gen_loss)
                #print(gen_obs[1,:], np.sum(gen_obs, axis=1))


    # Sampling helper function for evaluation
    def sampler(self):
        z_sample = np.random.normal(0., 1.0, size=(self.n_samples, self.latent_dim))
        prediction = self.generator.predict(z_sample).transpose()
        samples = np.zeros((self.input_dim_crit, self.n_samples))
        samples[:self.numerical_col_n,:]=prediction[:self.numerical_col_n,:]
        for idx in range(len(self.categories_cum)-1):
            idx_i = self.numerical_col_n+self.categories_cum[idx] # Initial index
            idx_f = self.numerical_col_n+self.categories_cum[idx+1] # Final index
            mask = np.argmax(prediction[idx_i:idx_f, :], axis=0) + idx_i
            for n in range(self.n_samples):
                samples[mask[n], n] = 1
        return samples
    
    # VAE evaluation
    def wgangp_evaluate(self, used_metric='MAE'):
        '''
        COMMENT THE CODE
        '''
        # Fit the model
        self.wgangp_fit()
        
        # Evaluate it
        self.samples = self.sampler()
        self.wgangp_df = samples_to_df(self.samples, print_duplicates=False)
        test_df = samples_to_df(self.data_test.transpose(), print_duplicates=False)

        # Numerical bin creator 
        for var in numerical:
            test_df[var], bins = pd.cut(test_df[var], bins=5,  retbins=True)
            self.wgangp_df[var] = pd.cut(self.wgangp_df[var], bins=bins)

        agg_vars = categorical # Variables we are using to aggregate and evaluate, change as needed 
        ##### Count creator
        self.wgangp_df['count'] = 1
        self.wgangp_df = self.wgangp_df.groupby(agg_vars, observed=True).count()
        self.wgangp_df /= self.wgangp_df['count'].sum()

        test_df['count'] = 1
        test_df = test_df.groupby(agg_vars, observed=True).count()
        test_df /= test_df['count'].sum()

        ##### Merge and difference
        real_and_sampled = pd.merge(test_df, self.wgangp_df, suffixes=['_real', '_sampled'], on=categorical, how='outer') # on= all variables
        real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)
        real_and_sampled['diff'] = real_and_sampled.count_real-real_and_sampled.count_sampled
        diff = np.array(real_and_sampled['diff'])
        
        metrics = {}
        metrics['MAE'] = np.mean(abs(diff))
        metrics['MSE'] = np.mean(diff**2)
        metrics['RMSE'] = np.sqrt(np.mean(diff**2))
        print('Evaluating with {}'.format(used_metric))
        print('MAE:{}, MSE:{}, RMSE:{}'.format(metrics['MAE'], metrics['MSE'], metrics['RMSE']))  
        
        return metrics[used_metric]

In [ ]:
epochs_WGANGP = 5000
wgangp_latent_dim = 100
prueba_WGANGP = WGANGP(train=x_train, test=x_test, numerical_col_n=numerical_col_n,
             categorical_col_n = categorical_col_n, categories_n = categories_n, 
             categories_cum = categories_cum, # Data
             intermediate_dim_gen=1024, latent_dim=wgangp_latent_dim, n_hidden_layers_gen=4, # Generator architecture 
             intermediate_dim_crit=1024, n_hidden_layers_crit=5, # Critic architecture 
             batch_size=64, epochs=epochs_WGANGP, gen_learn_rate=9e-05, 
             crit_learn_rate=6e-05, lmda=10, nCritic=5) # Training 
prueba_WGANGP.wgangp_evaluate()
# gen_learn_rate  
# crit_learn_rate
# latent_dim 
# n_hidden_layers_gen 
# n_hidden_layers_crit 
# intermediate_dim_gen 
# intermediate_dim_crit 
# batch_size 

# The best so far: [1e-05, 1e-02, 2.00e+01, 4.00e+00, 3.00e+00, 5.12e+02, 1.28e+02, 8.00e+00]

# GAN main (working)

In [ ]:
class GAN(): # Training based on https://github.com/eriklindernoren/Keras-GAN/blob/master/gan/gan.py
    
    def __init__(self, train, test, numerical_col_n, categorical_col_n, categories_n, categories_cum, # Data
                     intermediate_dim_gen=256, latent_dim=100, n_hidden_layers_gen=4, # Architecture Generator
                     intermediate_dim_dis=128, n_hidden_layers_dis=2, # Architecture Generator
                     batch_size=64, epochs=50, validation_split=0.2, gen_learn_rate=0.001, dis_learn_rate=0.001): # Training
        
        # Data parameters
        self.data_train = train
        self.data_test = test
        
        self.numerical_col_n = numerical_col_n # Scalar
        self.categorical_col_n = categorical_col_n # Scalar
        self.categories_n = categories_n # List of scalars
        self.categories_cum = categories_cum # List of scalars
        
        # Architecture parameters
        self.input_dim_dis = train.shape[1]
        self.latent_dim = latent_dim # Input dimension for generator
        self.intermediate_dim_dis = intermediate_dim_dis
        self.intermediate_dim_gen = intermediate_dim_gen
        self.n_hidden_layers_dis = n_hidden_layers_dis
        self.n_hidden_layers_gen = n_hidden_layers_gen
        
        # Training parameters
        self.epochs = epochs
        self.batch_size = batch_size
        self.validation_split = validation_split
        self.dis_learn_rate = dis_learn_rate
        self.gen_learn_rate = gen_learn_rate
        self.dis_opt = keras.optimizers.Adam(lr=self.dis_learn_rate)
        self.gen_opt = keras.optimizers.Adam(lr=self.gen_learn_rate)
        self.loss = 'binary_crossentropy'
        
        # Sampling parameters
        self.n_samples = test.shape[0]
        
        # Model creation
        self.create_gan()
        
        # Session variables
        #tf.reset_default_graph()
        sess = tf.InteractiveSession() # Start tf session so we can run code.
        K.set_session(sess) # Connect keras to the created session.
        

    # Generator architecture
    def create_generator(self):
        '''
        This is the generator architecture, 

        params: 
        n_hidden_layers_gen: number of hidden layers
        intermediate_dim_gen: value of the number of neurons for the first intermediate layer. 
            They increase in a factor of 2 (N*2). 
        latent_dim: dimension of latent space taken as input for the generator
        
        output:
        The function outputs the generator model in keras. The architecture is defined by the user when
        the whole GAN class is initialized.
        '''
            
        gen_input = Input(shape=(self.latent_dim,), name='gen_input')
        
        # Make the generator intermediate dimensions as it should be first
        self.intermediate_dim_gen = int(self.intermediate_dim_gen/2**(self.n_hidden_layers_gen-1))
        
        # Intermediate layers
        for _ in range(self.n_hidden_layers_gen):
            if _==0: # The first one takes the inputs as input
                intermediate = Dense(self.intermediate_dim_gen, name= 'generator_hidden_{}'.format(_), kernel_initializer='he_uniform', activation='relu')(gen_input)
            else: # After the first one, the network takes the intermediate layers as input
                intermediate = Dense(self.intermediate_dim_gen, name= 'generator_hidden_{}'.format(_), kernel_initializer='he_uniform', activation='relu')(intermediate)
            self.intermediate_dim_gen *= 2 # Update the value of the number of neurons

        # Final layer
        # Categorical decode
        x_decoded_mean_cat = [Dense(categories_n[cat], activation='softmax')(intermediate) 
                              for cat in range(len(self.categories_n))]

        if numerical_col_n > 0: # If there are numerical variables, concatenate both
            x_decoded_mean_num = Dense(self.numerical_col_n)(intermediate) # Numerical decode
            gen_output = concatenate([x_decoded_mean_num] + x_decoded_mean_cat, name='gen_output')
        else: # If there are no numerical variables only include the categorical output layer
            gen_output = concatenate(x_decoded_mean_cat, name='gen_output')

        return Model(inputs=gen_input, outputs=gen_output)
    
    # Discriminator architecture
    def create_discriminator(self):
        '''
        COMMENT THE CODE
        '''

        dis_input = Input(shape=(self.input_dim_dis,), name='dis_input')
        
        # Intermediate layers
        for _ in range(self.n_hidden_layers_dis):
            if _==0: # The first one takes the inputs as input
                intermediate = Dense(self.intermediate_dim_dis, name= 'discriminator_hidden_{}'.format(_), kernel_initializer='he_uniform', activation='relu')(dis_input)
            else: # After the first one, the network takes the intermediate layers as input
                intermediate = Dense(self.intermediate_dim_dis, name= 'discriminator_hidden_{}'.format(_), kernel_initializer='he_uniform', activation='relu')(intermediate)
            self.intermediate_dim = int(self.intermediate_dim_dis/2) # Update the value of the number of neurons
        
        dis_output = Dense(1, activation='sigmoid', name='dis_output')(intermediate)
        
        return Model(dis_input, dis_output)
    
    # GAN creation
    def create_gan(self):
        '''
        COMMENT THE CODE
        '''
        
        # Build and compile the discriminator
        self.discriminator = self.create_discriminator()
        self.discriminator.compile(loss=self.loss, optimizer=self.dis_opt, metrics=['accuracy'])

        # Build the generator
        self.generator = self.create_generator()

        # The generator takes noise as input and generates observations
        z = Input(shape=(self.latent_dim,))
        generated = self.generator(z)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The discriminator takes generated observations as input and discriminates
        guess = self.discriminator(generated)

        # The combined model  (stacked generator and discriminator)
        # Trains the generator to fool the discriminator
        self.gan = Model(z, guess)
        self.gan.compile(loss=self.loss, optimizer=self.gen_opt) 
        

    def gan_fit(self):
        '''
        COMMENT THE CODE
        '''

        # Adversarial ground truths
        valid = np.ones((self.batch_size, 1))
        fake = np.zeros((self.batch_size, 1))

        # Save the generator and discriminator losses and accuracies
        self.gen_loss = []
        self.dis_loss = []
        self.dis_acc = []
        
        for epoch in range(self.epochs):
            
            # ---------------------
            #  Train Discriminator
            # ---------------------

            # Select a random batch of observations
            idx = np.random.choice(self.data_train.shape[1], self.batch_size, replace=False)
            obs = self.data_train[idx]

            noise = np.random.normal(0, 1, (self.batch_size, self.latent_dim))

            # Generate a batch of new images
            gen_obs = self.generator.predict(noise)

            # Train the discriminator
            dis_loss_real = self.discriminator.train_on_batch(obs, valid)
            dis_loss_fake = self.discriminator.train_on_batch(gen_obs, fake)
            self.dis_loss.append(0.5 * np.add(dis_loss_real[0], dis_loss_fake[0]))
            self.dis_acc.append(0.5 * np.add(dis_loss_real[1], dis_loss_fake[1]))

            # ---------------------
            #  Train Generator
            # ---------------------

            noise = np.random.normal(0, 1, (self.batch_size, self.latent_dim))

            # Train the generator (to have the discriminator label samples as valid)
            self.gen_loss.append(self.gan.train_on_batch(noise, valid))
            
            if epoch%100==0:
                # Plot the progress
                print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, self.dis_loss[-1], 100*self.dis_acc[-1], self.gen_loss[-1]))
                #print (dis_loss_real[0], dis_loss_fake[0], gen_loss)
                #print(gen_obs[1,:], np.sum(gen_obs, axis=1))

    # Sampling helper function for evaluation
    def sampler(self):
        z_sample = np.random.normal(0., 1.0, size=(self.n_samples, self.latent_dim))
        prediction = self.generator.predict(z_sample).transpose()
        samples = np.zeros((self.input_dim_dis, self.n_samples))
        samples[:self.numerical_col_n,:]=prediction[:self.numerical_col_n,:]
        for idx in range(len(self.categories_cum)-1):
            idx_i = self.numerical_col_n+self.categories_cum[idx] # Initial index
            idx_f = self.numerical_col_n+self.categories_cum[idx+1] # Final index
            mask = np.argmax(prediction[idx_i:idx_f, :], axis=0) + idx_i
            for n in range(self.n_samples):
                samples[mask[n], n] = 1
        return samples
    
    # GAN evaluation
    def gan_evaluate(self, used_metric='MAE'):
        '''
        COMMENT THE CODE
        '''
        # Fit the model
        self.gan_fit()
        
        # Evaluate it
        self.samples = self.sampler()
        self.gan_df = samples_to_df(self.samples, print_duplicates=False)
        test_df = samples_to_df(self.data_test.transpose(), print_duplicates=False)

        # Numerical bin creator 
        for var in numerical:
            test_df[var], bins = pd.cut(test_df[var], bins=5,  retbins=True)
            self.gan_df[var] = pd.cut(self.gan_df[var], bins=bins)

        agg_vars = categorical # Variables we are using to aggregate and evaluate, change as needed 
        ##### Count creator
        self.gan_df['count'] = 1
        self.gan_df = self.gan_df.groupby(agg_vars, observed=True).count()
        self.gan_df /= self.gan_df['count'].sum()

        test_df['count'] = 1
        test_df = test_df.groupby(agg_vars, observed=True).count()
        test_df /= test_df['count'].sum()

        ##### Merge and difference
        real_and_sampled = pd.merge(test_df, self.gan_df, suffixes=['_real', '_sampled'], on=categorical, how='outer') # on= all variables
        real_and_sampled = real_and_sampled[['count_real', 'count_sampled']].fillna(0)
        real_and_sampled['diff'] = real_and_sampled.count_real-real_and_sampled.count_sampled
        diff = np.array(real_and_sampled['diff'])
        
        metrics = {}
        metrics['MAE'] = np.mean(abs(diff))
        metrics['MSE'] = np.mean(diff**2)
        metrics['RMSE'] = np.sqrt(np.mean(diff**2))
        print('Evaluating with {}'.format(used_metric))
        print('MAE:{}, MSE:{}, RMSE:{}'.format(metrics['MAE'], metrics['MSE'], metrics['RMSE']))  
        
        return metrics[used_metric]

In [ ]:
epochs_GAN = 20000
prueba_GAN = GAN(train=x_train, test=x_test, numerical_col_n=numerical_col_n,
             categorical_col_n = categorical_col_n, categories_n = categories_n, 
             categories_cum = categories_cum, # Data
             intermediate_dim_gen=1024, latent_dim=100, n_hidden_layers_gen=5, # Generator architecture 
             intermediate_dim_dis=1024, n_hidden_layers_dis=5, # Discriminator architecture 
             batch_size=64, epochs=epochs_GAN, gen_learn_rate=0.00001, dis_learn_rate=0.00001) # Training 
prueba_GAN.gan_evaluate()